In [1]:
import pandas as pd 
import numpy as np
import os
from math import ceil
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMolecule
import re

In [2]:
import sys
sys.path.append('../evaluation/')

In [3]:
from utils_eval import build_pdb_dict

In [4]:
# root_path = 'pharmacophore_specific_eval/1ywn'  # 0 0 
# root_path = 'pharmacophore_specific_eval/4ara' # 0.48 0.49
# root_path = 'pharmacophore_specific_eval/1lhn'  # 0.08 0.08
# root_path = 'pharmacophore_specific_eval/4b84'  # 0.57 0.70
# root_path = 'pharmacophore_specific_eval/5ni6'  # 0.07 0.08
# root_path = 'pharmacophore_specific_eval/5idn'  # 0.23 0.25 
# root_path = 'pharmacophore_specific_eval/5ni2'  # 0.13 0.15
# root_path = 'pharmacophore_specific_eval/4ijw'  # 0.36 0.39
# root_path = 'pharmacophore_specific_eval/3ey4'  # 0.49 0.45
# root_path = 'pharmacophore_specific_eval/3r58'  # 0.70 0.70
# root_path = 'pharmacophore_specific_eval/4k1l'  # 0.58 0.62



# aromatic = True

In [4]:
# ligand_name = '1ywn_A_rec_1ywn_lif_lig_tt_docked_0'  # 0 0
# ligand_name = '4b81_B_rec_4ara_c56_lig_tt_min_0'  # 0.41 0.51
# ligand_name = '1lhu_A_rec_1lhn_aon_lig_tt_min_0'  # 0.07 0.11
# ligand_name = '4b84_A_rec_4b84_z5k_lig_tt_min_0'  # 0.60 0.61
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'  
# ligand_name = '5hbh_A_rec_5idn_6a7_lig_tt_min_0'
# ligand_name = '5aen_A_rec_5ni2_dj3_lig_tt_min_0'
# ligand_name = '3d3e_B_rec_4ijw_1eq_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_3ey4_352_lig_tt_docked_12'
# ligand_name = '4h7c_A_rec_3r58_nps_lig_tt_min_0'  # 0.64 0.70
# ligand_name = '3ey4_C_rec_4k1l_sff_lig_tt_min_0'  # 0.59 0.59 

# ligand_name = '3hfg_A_rec_3hfg_17r_lig_tt_min_0'
# ligand_name = '3ey4_C_rec_5pgu_8k4_lig_tt_min_0'  # 0.11 0.10
# ligand_name = '5ni2_A_rec_5aen_dp8_lig_tt_min_0'
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_2ilt_nn1_lig_tt_min_0'  # 0.16 0.20 
# ligand_name = '4bnn_A_rec_4bnm_tcu_lig_tt_min_0'
# ligand_name = '4yvx_A_rec_1s2a_imn_lig_tt_min_0'
# ligand_name = '3czr_B_rec_4bb5_hd2_lig_tt_min_0'
# ligand_name = '3czr_A_rec_2ilt_nn1_lig_tt_min_0'
# ligand_name = '4b84_A_rec_4btl_5gz_lig_tt_min_0'
# ligand_name = '3fue_A_rec_3ftv_11x_lig_tt_min_0'  # 0.04 0.05
# ligand_name = '4bnk_B_rec_4d44_ja3_lig_tt_min_0'  # 0.26 0.27
# ligand_name = '3qqp_B_rec_4k1l_sff_lig_tt_min_0'
# ligand_name = '4wdu_A_rec_3r94_flr_lig_tt_min_0'  # 0.37 0.27
# ligand_name = '4alj_A_rec_4bnf_pv4_lig_tt_docked_0'  # 0.24 0.24
# ligand_name = '1ejj_A_rec_1o99_2pg_lig_tt_docked_0'
# ligand_name = '4h96_B_rec_4h95_06u_lig_tt_min_0'  # 0.40 0.42
# ligand_name = '2oa6_D_rec_4kvi_1sv_lig_tt_min_0'  # 0.36 0.41
# ligand_name = '4hx5_B_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgy_D_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgv_A_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '4kux_B_rec_4kvw_jf4_lig_tt_min_0'  # 0.61 0.63
# ligand_name = '4bnk_B_rec_4bnl_w1g_lig_tt_min_0'
# ligand_name = '4b82_B_rec_4ara_c56_lig_tt_min_0'  # 0.42 0.53
# ligand_name = '4pm0_A_rec_4y2b_epk_lig_tt_min_0'
# ligand_name = '4ms6_A_rec_3fuh_5h1_lig_tt_min_0'  # 0.02 0.02
# ligand_name = '5ivg_B_rec_4kvy_jf1_lig_tt_docked_9'  # 0.61 0.69
# ligand_name = '3r8h_A_rec_3r8h_zom_lig_tt_min_0'
# ligand_name = '1aoe_A_rec_3qlr_qlr_lig_tt_docked_4'  # 0.19 0.28
ligand_name = '2ilt_A_rec_3ey4_352_lig_tt_min_0'  # 0.70 0.71
# ligand_name = '2g1q_A_rec_1x88_nat_lig_tt_min_0'  # 0.59 0.59
# ligand_name = '5imp_B_rec_4kwd_jf2_lig_tt_min_0'  # 0.54 0.55
# ligand_name = '4d45_A_rec_4ali_tcl_lig_tt_min_0'  # 0.57 0.52
# ligand_name = '5ivg_B_rec_4kvw_jf4_lig_tt_min_0'
# ligand_name = '2rbe_C_rec_3byz_h11_lig_tt_min_0'
# ligand_name = '2whr_A_rec_3zlv_hi6_lig_tt_min_0'
# ligand_name = '4hmn_A_rec_3ufy_npx_lig_tt_min_0'
# ligand_name = '4h7c_A_rec_3r6i_jms_lig_tt_min_0'

# folder_name = ligand_name[ligand_name.rfind('rec')+4:ligand_name.rfind('rec')+8]
bridge_type = 'vp'
root_path = os.path.join('structure_based', bridge_type, ligand_name)

aromatic = True

In [5]:
log_path = os.path.join(root_path, 'logs')
log_path = log_path+'_aromatic' if aromatic else log_path

raw_data_path = '../../data/cleaned_crossdocked_data/raw'

log_files = os.listdir(log_path)
pdb_dict, pdb_rev_dict = build_pdb_dict(raw_data_path)

In [6]:
def get_minimized_affinity(filename, mode='ref'):
    if mode == 'ref':
        with open(filename, 'r') as file:
            for line in file:
                if '<minimizedAffinity>' in line:
                    # The next line after '<minimizedAffinity>' contains the value
                    return float(next(file))
            
    else:
        with open(filename, 'r') as f:
            lines = f.readlines()
            lines = lines[19:28]
#             print(len(lines))
            n = len(lines)
            arr = np.zeros(n)
            cnn_arr = np.zeros(n)
            # print(filename)
            for i in range(n):
                arr[i] = float(lines[i][11:17])
                cnn_arr[i] = float(lines[i][34:42])
#             print(arr)
            try:
                score = np.min(arr)
                cnn_score = np.min(cnn_arr)
            except:
                score = None
                cnn_score = None
                print(f'{filename} failed')
                print('its scores: ', arr)
        return score, cnn_score

In [7]:
def get_affinity_best_pose(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        lines = lines[19:28]
        try:
            score = float(lines[0][11:17])
            cnn_score = float(lines[0][34:42])
            # cnn_score = float(lines[0][24:30])
        except:
            score = None
            cnn_score = None
        return score, cnn_score

In [8]:
# ligand_filename = '1ywn_A_rec_1ywn_lif_lig_tt_docked_0.sdf'
# ligand_filename = '4b81_B_rec_4ara_c56_lig_tt_min_0.sdf'
# ligand_filename = '1lhu_A_rec_1lhn_aon_lig_tt_min_0.sdf'
# ligand_filename = '4b84_A_rec_4b84_z5k_lig_tt_min_0.sdf'
# ligand_filename = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0.sdf'
# ligand_filename = '5hbh_A_rec_5idn_6a7_lig_tt_min_0.sdf'
# ligand_filename = '5aen_A_rec_5ni2_dj3_lig_tt_min_0.sdf'
# ligand_filename = '3d3e_B_rec_4ijw_1eq_lig_tt_min_0.sdf'
# ligand_filename = '4yyz_B_rec_3ey4_352_lig_tt_docked_12.sdf'
# ligand_filename = '4h7c_A_rec_3r58_nps_lig_tt_min_0.sdf'
# ligand_filename = '3ey4_C_rec_4k1l_sff_lig_tt_min_0.sdf'

ligand_filename = ligand_name + '.sdf'
pdb_folder = pdb_rev_dict[ligand_filename]
ref_file = os.path.join(raw_data_path, pdb_folder, ligand_filename)
ref_score = get_minimized_affinity(ref_file, 'ref')

score_dict = {}
for log_file in tqdm(log_files):
    filename = log_file.split('.')[0]
#     print(log_file)
    gen_score, cnn_score = get_minimized_affinity(os.path.join(log_path, log_file), 'gen')
    # gen_score, cnn_score = get_affinity_best_pose(os.path.join(log_path, log_file))
    score_dict[filename] = [gen_score, cnn_score, ref_score]

 13%|█████████▌                                                                | 65/500 [00:00<00:00, 643.80it/s]

structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_99.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_135.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_468.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_20.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_206.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_117.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_326.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_476.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_332.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_306.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_329.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_133.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/lo

 39%|████████████████████████████▏                                            | 193/500 [00:00<00:00, 616.07it/s]

structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_491.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_126.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_208.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_211.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_122.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_473.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_289.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_263.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_108.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_217.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_10.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_402.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/l

 63%|█████████████████████████████████████████████▉                           | 315/500 [00:00<00:00, 582.59it/s]

structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_367.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_184.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_393.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_287.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_92.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_319.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_147.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_25.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_31.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_445.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_86.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_152.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs

 88%|████████████████████████████████████████████████████████████████         | 439/500 [00:00<00:00, 600.05it/s]

structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_314.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_414.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_309.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_93.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_167.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_242.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_294.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_0.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_149.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_109.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_137.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_89.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs

100%|█████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 600.14it/s]

structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_444.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_434.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_471.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_15.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_214.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_198.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_75.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_490.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_82.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_176.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_479.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/logs_aromatic/3ey4_143.log
structure_based/vp/2ilt_A_rec_3ey4_352_lig_tt_min_0/log

In [9]:
df = pd.DataFrame.from_dict(score_dict, orient='index', columns=['Binding affinity', 'CNN score', 'Reference'])
df

,Binding affinity,CNN score,Reference
3ey4_99,-8.28,0.5848,-7.42602
3ey4_135,-7.79,0.6368,-7.42602
3ey4_468,-7.83,0.6023,-7.42602
3ey4_20,-7.80,0.6264,-7.42602
3ey4_206,-7.87,0.6101,-7.42602
...,...,...,...
3ey4_479,-8.26,0.5973,-7.42602
3ey4_143,-8.22,0.6624,-7.42602
3ey4_478,-7.87,0.5929,-7.42602
3ey4_429,-7.96,0.5882,-7.42602


In [10]:
filtered_df = df.loc[df['Binding affinity']<df['Reference']]
filtered_df

,Binding affinity,CNN score,Reference
3ey4_99,-8.28,0.5848,-7.42602
3ey4_135,-7.79,0.6368,-7.42602
3ey4_468,-7.83,0.6023,-7.42602
3ey4_20,-7.80,0.6264,-7.42602
3ey4_206,-7.87,0.6101,-7.42602
...,...,...,...
3ey4_479,-8.26,0.5973,-7.42602
3ey4_143,-8.22,0.6624,-7.42602
3ey4_478,-7.87,0.5929,-7.42602
3ey4_429,-7.96,0.5882,-7.42602


In [11]:
sorted_df = filtered_df.sort_values(by='CNN score', ascending=False)
sorted_df

,Binding affinity,CNN score,Reference
3ey4_205,-8.25,0.7076,-7.42602
3ey4_190,-7.86,0.6905,-7.42602
3ey4_246,-8.32,0.6873,-7.42602
3ey4_231,-8.20,0.6871,-7.42602
3ey4_301,-7.89,0.6863,-7.42602
...,...,...,...
3ey4_499,-7.85,0.5715,-7.42602
3ey4_352,-7.84,0.5709,-7.42602
3ey4_440,-7.79,0.5670,-7.42602
3ey4_120,-7.82,0.5646,-7.42602


In [12]:
ba_res_filename = os.path.join(root_path, 'binding_affinity')
ba_res_filename = ba_res_filename+'_aromatic.csv' if aromatic else ba_res_filename+'.csv'

# sorted_df.to_csv(ba_res_filename)
# ba_res_filename

In [13]:
pp_match_csv = os.path.join(root_path, 'aromatic_scores.csv' if aromatic else 'basic_scores.csv')
pp_match_df = pd.read_csv(pp_match_csv, header=0, index_col=0)
pp_match_df = pp_match_df.rename(columns = {'score':'PP match score'})
pp_match_df

,PP match score
3ey4_38,0.818182
3ey4_314,0.818182
3ey4_145,0.818182
3ey4_352,0.818182
3ey4_385,0.818182
...,...
3ey4_76,0.818182
3ey4_425,0.818182
3ey4_66,0.818182
3ey4_468,0.818182


In [14]:
joint_df = df.join(pp_match_df)
joint_df

,Binding affinity,CNN score,Reference,PP match score
3ey4_99,-8.28,0.5848,-7.42602,0.818182
3ey4_135,-7.79,0.6368,-7.42602,0.818182
3ey4_468,-7.83,0.6023,-7.42602,0.818182
3ey4_20,-7.80,0.6264,-7.42602,0.818182
3ey4_206,-7.87,0.6101,-7.42602,0.818182
...,...,...,...,...
3ey4_479,-8.26,0.5973,-7.42602,0.818182
3ey4_143,-8.22,0.6624,-7.42602,0.818182
3ey4_478,-7.87,0.5929,-7.42602,0.818182
3ey4_429,-7.96,0.5882,-7.42602,0.818182


In [15]:
joint_res_fn = 'joint_res_aromatic.csv' if aromatic else 'joint_res.csv'
joint_df.to_csv(os.path.join(root_path, joint_res_fn))

In [16]:
sum(df['Binding affinity'] <= df['Reference']) / len(df)

1.0